In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

WEAVIATE_IP = os.getenv("WEAVIATE_IP")

## Connect

In [ ]:
import weaviate

client = weaviate.connect_to_custom(
    http_host={WEAVIATE_IP}, http_port="8080",  http_secure=False,
    grpc_host={WEAVIATE_IP}, grpc_port="50051", grpc_secure=False,
)

# Connect to the local instance deployed with Docker Compose
# client = weaviate.connect_to_local()

client.is_ready()

## Create Tenant-ready collection

In [ ]:
from weaviate.classes.config import Configure

if (client.collections.exists("PapersTen")):
    client.collections.delete("PapersTen")

client.collections.create(
    "PapersTen",

    vectorizer_config=Configure.Vectorizer.text2vec_aws(
        model="amazon.titan-embed-text-v1",
        # region="eu-central-1",
        region="us-east-1",
    ),
    
    generative_config=Configure.Generative.aws(
        region="us-east-1",
        model="cohere.command-r-plus-v1:0"
    ),

    multi_tenancy_config=Configure.multi_tenancy(True)
)

## List Tenants

In [ ]:
papers = client.collections.get("PapersTen")
papers.tenants.get()

## Close the client when done

In [ ]:
client.close()